## Evaluating a vectorial function on CPU and GPU

### CPU: plain and numpy

In [6]:
import numpy as np
from numba import njit, jit

# Python plain implementation w/ numba 
@njit
def grade2_vector(x, y, a, b, c):
    z = np.zeros(x.size)
    for i in range(x.size):
        z[i] = a*x[i]*x[i] + b*y[i] + c
    return z

# Numpy ufunc
def grade2_ufunc(x, y, a, b, c):
    return a*x**2 + b*y + c

# size of the vectors
size = 5_000_000

# allocating and populating the vectors
a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)
c_cpu = np.zeros(size)

a = 3.5
b = 2.8
c = 10

# Printing input values
#print(a_cpu)
#print(b_cpu)
# Random function in Numpy always use float64
print(a_cpu.dtype)

c_cpu = grade2_vector(a_cpu, b_cpu, a, b, c)


# Evaluating the time

# Numba Python: huge improvement, better that numpy code
%timeit -n 5 -r 2 grade2_vector(a_cpu, b_cpu, a, b, c)

# w/ a numpy ufunc manually coded
%timeit -n 5 -r 2 grade2_ufunc(a_cpu, b_cpu, a, b, c)

# using the general numpy ufunc 
%timeit -n 5 -r 2 a*a_cpu**2 + b*b_cpu + c

float64
16.4 ms ± 1.6 ms per loop (mean ± std. dev. of 2 runs, 5 loops each)
43 ms ± 11.4 ms per loop (mean ± std. dev. of 2 runs, 5 loops each)
28.6 ms ± 678 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)


### 3.2 A) Celda GPU con CuPy (con y sin copia)

In [7]:
import cupy as cp
from cupyx.profiler import benchmark  # para medir tiempos de GPU :contentReference[oaicite:0]{index=0}

# Misma ufunc pero pensada para trabajar con arrays de CuPy
def grade2_ufunc_gpu(x, y, a, b, c):
    return a * x**2 + b * y + c

# CASO 1: CONTANDO LA COPIA CPU -> GPU EN EL TIEMPO 

def run_cupy_with_copy():
    # copiamos los datos de NumPy (CPU) a CuPy (GPU) dentro de la función
    x_gpu = cp.asarray(a_cpu)
    y_gpu = cp.asarray(b_cpu)
    z_gpu = grade2_ufunc_gpu(x_gpu, y_gpu, a, b, c)
    # sincronizamos para que acabe el trabajo en la GPU antes de medir
    cp.cuda.Stream.null.synchronize()
    return z_gpu

bench_copy = benchmark(run_cupy_with_copy, (), n_repeat=10)
print("CuPy con copia CPU->GPU:")
print(bench_copy)


# CASO 2: SIN CONTAR LA COPIA (solo cálculo en GPU)

# Copiamos solo una vez fuera de la función de benchmark
a_gpu = cp.asarray(a_cpu)
b_gpu = cp.asarray(b_cpu)

def run_cupy_no_copy():
    z_gpu = grade2_ufunc_gpu(a_gpu, b_gpu, a, b, c)
    cp.cuda.Stream.null.synchronize()
    return z_gpu

bench_no_copy = benchmark(run_cupy_no_copy, (), n_repeat=10)
print("\nCuPy sin copia (solo cálculo en GPU):")
print(bench_no_copy)

CuPy con copia CPU->GPU:
run_cupy_with_copy  :    CPU: 14012.362 us   +/- 46.721 (min: 13937.715 / max: 14076.917) us     GPU-0: 14019.088 us   +/- 46.764 (min: 13942.816 / max: 14083.168) us

CuPy sin copia (solo cálculo en GPU):
run_cupy_no_copy    :    CPU:  4288.401 us   +/-  3.499 (min:  4285.424 / max:  4297.566) us     GPU-0:  4292.445 us   +/-  3.693 (min:  4289.280 / max:  4301.824) us


### 3.2 B) Celda GPU con Numba @vectorize(target='cuda')

In [8]:
from numba import vectorize, cuda

# ufunc de Numba que se ejecuta en la GPU
@vectorize(['float64(float64, float64, float64, float64, float64)'], target='cuda')
def grade2_vector_gpu(x, y, a, b, c):
    return a * x * x + b * y + c


# CASO 1: Numba con COPIA AUTOMÁTICA (CPU <-> GPU)

def run_numba_with_copy():
    # Pasamos arrays NumPy; Numba se encarga de copiar a la GPU y devolver a CPU
    z = grade2_vector_gpu(a_cpu, b_cpu, a, b, c)
    cuda.synchronize()
    return z

bench_numba_copy = benchmark(run_numba_with_copy, (), n_repeat=10)
print("Numba GPU con copia automática (CPU <-> GPU):")
print(bench_numba_copy)


# CASO 2: sin contar la copia (datos ya en la GPU)

# Copiamos primero a GPU como arrays de CuPy
a_gpu = cp.asarray(a_cpu)
b_gpu = cp.asarray(b_cpu)

def run_numba_no_copy():
    # Numba puede trabajar con arrays que implementan __cuda_array_interface__,
    # como los de CuPy, sin hacer copias extra. :contentReference[oaicite:1]{index=1}
    z = grade2_vector_gpu(a_gpu, b_gpu, a, b, c)
    cuda.synchronize()
    return z

bench_numba_no_copy = benchmark(run_numba_no_copy, (), n_repeat=10)
print("\nNumba GPU sin contar copia (solo cálculo en GPU):")
print(bench_numba_no_copy)

Numba GPU con copia automática (CPU <-> GPU):
run_numba_with_copy :    CPU: 18597.727 us   +/- 2238.902 (min: 15382.026 / max: 21410.432) us     GPU-0: 18606.054 us   +/- 2239.156 (min: 15389.888 / max: 21419.584) us

Numba GPU sin contar copia (solo cálculo en GPU):
run_numba_no_copy   :    CPU:  2355.255 us   +/- 977.990 (min:  1335.105 / max:  3553.797) us     GPU-0:  2361.178 us   +/- 978.272 (min:  1340.832 / max:  3560.864) us


### 3.2 C) Comentario de resultados (CPU vs GPU con CuPy y Numba)


En este ejercicio se ha evaluado la función vectorial `z = a*x*x + b*y + c` usando vectores de 5 millones de elementos, comparando la ejecución en CPU y GPU con distintas librerías.

---

### Resultados en CPU (tiempos medios)

- `grade2_vector` con Numba (`@njit`): **16.4 ms ± 1.6 ms**  
- `grade2_ufunc` manual en NumPy: **43 ms ± 11.4 ms**  
- Expresión NumPy general: **28.6 ms ± 0.7 ms**

La versión con Numba en CPU es claramente la más eficiente, muy por delante de las dos variantes en NumPy.

---

### Resultados en GPU con CuPy

- Con copia CPU->GPU incluida: **≈ 14.0 ms**  
- Sin copia (datos ya en GPU): **≈ 4.3 ms**

Cuando se incluye la copia de datos, el tiempo total es similar al de la mejor versión en CPU, lo que muestra que la transferencia de datos puede compensar la aceleración de la GPU.  
Sin embargo, cuando se mide solo el cálculo con los datos ya en la GPU, CuPy es claramente más rápido que cualquier variante en CPU.

---

### Resultados en GPU con Numba (`@vectorize(target='cuda')`)

- Con copia automática CPU<->GPU: **≈ 18.6 ms**  
- Sin copia (datos en GPU): **≈ 2.4 ms**

La copia automática hace que el tiempo total sea peor que en CPU.  
En cambio, cuando los datos ya están en la GPU, Numba obtiene el mejor tiempo de todos, alrededor de 2.4 ms.

---

### Conclusiones

- La CPU con Numba (`@njit`) mejora notablemente a NumPy, pero la GPU puede superar claramente a la CPU si se minimizan las copias de datos.
- CuPy y Numba en GPU muestran tiempos muy reducidos cuando los arrays residen ya en la GPU.
- La mejor ejecución obtenida ha sido la de **Numba en GPU sin copia**, con unos 2.4 ms.
- Si las copias CPU↔GPU se incluyen en la medida, la ganancia desaparece o incluso empeora respecto a la CPU, lo que demuestra que el coste de transferencia es crítico en problemas de gran tamaño.
